<a href="https://colab.research.google.com/github/hargurjeet/MachineLearning/blob/master/ML_using_Kfolds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing dataset from Kaggle

In [ ]:
!pip install opendatasets --upgrade --quiet

In [10]:
import numpy as np
import pandas as pd
import opendatasets as od
from sklearn import model_selection

In [4]:
od.download('https://www.kaggle.com/c/30-days-of-ml/data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: hargurjeet
Your Kaggle Key: ··········


100%|██████████| 66.0M/66.0M [00:00<00:00, 127MB/s]



Extracting archive ./30-days-of-ml/30-days-of-ml.zip to ./30-days-of-ml


# Splitting the data to kfold

In [45]:
df_train = pd.read_csv("/content/30-days-of-ml/train.csv")

In [46]:
df_train["kfold"] = -1

In [47]:
kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_indicies, valid_indicies) in enumerate(kf.split(X=df_train)):
    df_train.loc[valid_indicies, "kfold"] = fold

In [52]:
df_train.sample(5)

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target,kfold
180612,301182,B,B,A,A,B,B,A,E,C,F,0.424574,0.425265,0.288815,0.299603,0.905591,0.403610,0.603136,0.285114,0.259144,0.681920,0.171736,0.346228,0.219408,0.494149,7.125354,0
152515,254225,A,A,A,C,B,B,A,E,C,I,0.469408,0.418555,0.727808,0.274424,0.491125,0.141452,0.481424,0.869121,0.258237,0.290514,0.272234,0.556597,0.605068,0.458564,8.787194,4
239451,399317,A,B,A,C,B,D,A,E,C,J,0.771602,0.260283,0.054124,0.310650,0.285691,0.696135,0.582873,0.496103,0.121963,0.593459,0.450313,0.346291,0.840787,0.442029,8.840280,2
198798,331500,A,A,B,C,B,B,A,E,E,I,0.436875,0.364887,0.531707,0.594590,0.473787,0.399627,0.156658,0.840842,0.347336,0.317794,0.066265,0.286467,0.402870,0.796672,7.886309,4
174752,291379,A,A,A,A,B,B,A,E,G,C,0.399640,0.005195,0.620999,0.138845,0.287439,0.285327,0.843390,0.351287,0.271202,0.360853,0.535909,0.549679,0.167316,0.231910,6.968845,2


In [53]:
df_train.kfold.value_counts()

4    60000
3    60000
2    60000
1    60000
0    60000
Name: kfold, dtype: int64

In [48]:
df_train.to_csv("train_folds.csv", index=False)